## Importing the libraries

In [2]:
import os 
import re
import pandas as pd
import numpy as np

## Connecting to the folder containing the csv files, combining them into single dataframe for years 2018-2020 

**Important Note-**

-All the csv files of trip data are split into two folders because of huge size to process together. 

-This spliting is done on the basis of file names. All the csv file names like 'Divvy_Trips_2015_07.csv' are stored in one      folder and the remaining csv files having names like '202004-divvy-tripdata.csv' are stored in another folder. 

-This is done for convenience of handling the data.

Folder 1 = All csv files having names like '202004-divvy-tripdata.csv'

Folder 2 = All csv files having names like 'Divvy_Trips_2015_07.csv'


In [3]:
folder_path = r'insert the folder path containing all trip data files of type-Divvy_Trips_2015_07.csv '  # Replace with your actual folder path
# Initialize an empty list to hold DataFrames
df_list = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv') and re.search( r'(?<!\d)(2018|2019|2020)(?!\d)', filename):
        file_path = os.path.join(folder_path, filename)
        try:
            df = pd.read_csv(file_path)
            df_list.append(df)
        except Exception as e:
            print(f"Error reading {filename}: {e}")
# Combine all DataFrames into one
if df_list:
    combined_df = pd.concat(df_list, ignore_index=True)
    print(combined_df.head())
else:
    print("⚠️ No matching files found or all failed to load.")
#viewing the dataset for correct columns
combined_df.columns

   01 - Rental Details Rental ID 01 - Rental Details Local Start Time   
0                     17536702.0                  2018-01-01 00:12:00  \
1                     17536703.0                  2018-01-01 00:41:35   
2                     17536704.0                  2018-01-01 00:44:46   
3                     17536705.0                  2018-01-01 00:53:10   
4                     17536706.0                  2018-01-01 00:53:37   

  01 - Rental Details Local End Time  01 - Rental Details Bike ID   
0                2018-01-01 00:17:23                       3304.0  \
1                2018-01-01 00:47:52                       5367.0   
2                2018-01-01 01:33:10                       4599.0   
3                2018-01-01 01:05:37                       2302.0   
4                2018-01-01 00:56:40                       3696.0   

  01 - Rental Details Duration In Seconds Uncapped   
0                                            323.0  \
1                                     

Index(['01 - Rental Details Rental ID', '01 - Rental Details Local Start Time',
       '01 - Rental Details Local End Time', '01 - Rental Details Bike ID',
       '01 - Rental Details Duration In Seconds Uncapped',
       '03 - Rental Start Station ID', '03 - Rental Start Station Name',
       '02 - Rental End Station ID', '02 - Rental End Station Name',
       'User Type', 'Member Gender',
       '05 - Member Details Member Birthday Year', 'trip_id', 'start_time',
       'end_time', 'bikeid', 'tripduration', 'from_station_id',
       'from_station_name', 'to_station_id', 'to_station_name', 'usertype',
       'gender', 'birthyear', 'ride_id', 'rideable_type', 'started_at',
       'ended_at', 'start_station_name', 'start_station_id',
       'end_station_name', 'end_station_id', 'start_lat', 'start_lng',
       'end_lat', 'end_lng', 'member_casual'],
      dtype='object')

## Preprocessing the dataset and viewing the columns,size and data types  

In [ ]:
combined_df.head()
combined_df.shape[0]
combined_df.dtypes
combined_df.columns

## Data Mapping and Integration into common format to prevent information loss
   - Columns in dataset gave us the insight that records were stored in three different column formats and                            this insights led us to segregate the data followed by mapping and then integrating together into single columns 
   - This helped to prevent information loss and millions of records were uniqfied in common name format
   - Entire dataset was split into 3 subsets and then data trasformations and cleaning was performed on each subset
   - Therefore these subsets were concatenated together into a single dataframe without losing much information

In [68]:
#Calculate the Na values to give distribution across columns
combined_df.isna().sum() 

#subset 1
dfa=combined_df[['01 - Rental Details Rental ID', '01 - Rental Details Local Start Time',
       '01 - Rental Details Local End Time', '01 - Rental Details Bike ID',
       '01 - Rental Details Duration In Seconds Uncapped',
       '03 - Rental Start Station ID', '03 - Rental Start Station Name',
       '02 - Rental End Station ID', '02 - Rental End Station Name',
       'User Type', 'Member Gender']]
#subset 2
dfb=combined_df[['trip_id', 'start_time',
       'end_time', 'bikeid', 'tripduration', 'from_station_id',
       'from_station_name', 'to_station_id', 'to_station_name', 'usertype']]
#subset 3
dfc=combined_df[[ 'ride_id', 'rideable_type', 'started_at',
       'ended_at', 'start_station_name', 'start_station_id',
       'end_station_name', 'end_station_id', 'start_lat', 'start_lng',
       'end_lat', 'end_lng', 'member_casual']]

#Removal of Na values in each subset 
a=dfa.dropna()
b=dfb.dropna()
c=dfc.dropna()

#Column transformations and renaming of columns in these subsets
a=a.rename(columns={'01 - Rental Details Rental ID':'ride_id','01 - Rental Details Local Start Time':'started_at',
        '01 - Rental Details Local End Time':'ended_at','01 - Rental Details Bike ID':'bikeid',
        '01 - Rental Details Duration In Seconds Uncapped':'trip_duration',
        '03 - Rental Start Station ID':'start_station_id','03 - Rental Start Station Name':'start_station_name',
        '02 - Rental End Station ID':'end_station_id','02 - Rental End Station Name':'end_station_name',
        'User Type':'member_casual','Member Gender':'gender','05 - Member Details Member Birthday Year':'birth_year'})

b=b.rename(columns={'trip_id':'ride_id','start_time':'started_at','end_time':'ended_at',
            'tripduration':'trip_duration','from_station_id':'start_station_id',
            'from_station_name':'start_station_name','to_station_id':'end_station_id',
            'to_station_name':'end_station_name','usertype':'member_casual'})

#Changing data format of certain columns of subset 3
c[['started_at','ended_at']]=pd.to_datetime(c[['started_at','ended_at']].stack(),
                           infer_datetime_format=True,format='%m/%d/%Y %H:%M').unstack()

#Adding new column in subset 3 and by making transformation on existing ones
c['trip_duration']=(c.ended_at-c.started_at).dt.total_seconds()/3600
c['trip_duration'].dtype

#Data integration of certain values in column into single category for subset 1 and 2
a['member_casual']= np.where(a['member_casual']=='Subscriber', 'member', 'casual')
b['member_casual']= np.where(b['member_casual']=='Subscriber', 'member', 'casual')

#Data Integration into single dataframe by concatenating all subsets
clean=pd.concat([a,b,c], ignore_index=True)

#Changing data types of columns for optimizing memory 
clean=clean.astype({'ride_id':'string','started_at':'datetime64[ns]','ended_at':'datetime64[ns]','bikeid':'string',
                    'start_station_name':'string','end_station_name':'string',
                      'start_station_id':'string','end_station_id':'string',
                     'rideable_type':'category','gender':'category'})
clean['trip_duration'] = clean['trip_duration'].str.replace(',', '', regex=False).astype(float)

#Sorting the cleaned dataset by dates in ascending order
clean = clean.sort_values(by=['started_at', 'ended_at'], ascending=True)
print(clean.head())

      ride_id          started_at            ended_at  bikeid  trip_duration   
0  17536702.0 2018-01-01 00:12:00 2018-01-01 00:17:23  3304.0          323.0  \
1  17536703.0 2018-01-01 00:41:35 2018-01-01 00:47:52  5367.0          377.0   
2  17536704.0 2018-01-01 00:44:46 2018-01-01 01:33:10  4599.0         2904.0   
3  17536705.0 2018-01-01 00:53:10 2018-01-01 01:05:37  2302.0          747.0   
4  17536706.0 2018-01-01 00:53:37 2018-01-01 00:56:40  3696.0          183.0   

  start_station_id           start_station_name end_station_id   
0             69.0       Damen Ave & Pierce Ave          159.0  \
1            253.0  Winthrop Ave & Lawrence Ave          325.0   
2             98.0   LaSalle St & Washington St          509.0   
3            125.0         Rush St & Hubbard St          364.0   
4            129.0    Blue Island Ave & 18th St          205.0   

                 end_station_name member_casual gender rideable_type   
0       Claremont Ave & Hirsch St        casual   

## Saving the cleaned dataset in your local

In [70]:
#cleaned data containing the trips from years 2018-2020 
clean.to_csv('insert the file path of csv file to store with name like-trips_18-20.csv')

## Importing the dataset from year- 2018-2020 and 2020-2023 

In [ ]:
import pandas as pd
#reading the trip data from years 2020-2023 
df1=pd.read_csv('insert the file path of csv file like- trips_20-23.csv')  # contains the trip data stored in csv file from previous jupyter notebook
#reading the trip data from years 2018-2020
df=pd.read_csv('insert the file path of csv file like trips_18-20.csv')   # contains the data stored in csv from year 2018-2020   

## Data pre-processing and profiling with correct mapping

In [43]:
df1=df1.astype({'ride_id':'string','started_at':'datetime64[ns]','ended_at':'datetime64[ns]',
                    'start_station_name':'string','end_station_name':'string',
                      'start_station_id':'string','end_station_id':'string',
                     'rideable_type':'category','member_casual':'category'})
print(df1.dtypes)

df=df.astype({'ride_id':'string',
                    'start_station_name':'string','end_station_name':'string',
                      'start_station_id':'string','end_station_id':'string',
                     'rideable_type':'category','member_casual':'category'})
print(df.dtypes)

Unnamed: 0                     int64
ride_id               string[python]
started_at            datetime64[ns]
ended_at              datetime64[ns]
bikeid                       float64
trip_duration                float64
start_station_id      string[python]
start_station_name    string[python]
end_station_id        string[python]
end_station_name      string[python]
member_casual               category
gender                        object
rideable_type               category
start_lat                    float64
start_lng                    float64
end_lat                      float64
end_lng                      float64
dtype: object

## Combining the datasets into single dataframe containing years 2018-2023

In [50]:
#combining both the datasets containing years:- 2018-2020 and 2020-2023
mdf=pd.concat([df,df1], ignore_index=True)
#adding columns for year and month
mdf['year']=mdf['ended_at'].dt.year
mdf['month']=mdf['ended_at'].dt.strftime('%B')
#Converting duration in seconds column into hours 
mdf['trip_duration']=(mdf.ended_at-mdf.started_at).dt.total_seconds()/3600
mdf['trip_duration'].isna().sum()

## Selecting the data for analysis containing years-2018-2021 

In [71]:
trips_4yr=mdf.query('2018 <= year <= 2021')
trips_4yr.shape[0]
trips_4yr['ride_id']='"'+ trips_4yr['ride_id'] + '"'
#saving the data in csv file for years 2018-2021
trips_4yr.to_csv('insert the file path for storing the combined data with names like- trips_4yr.csv ',index=False)